![CAT_DOG](img/bannière.png)

# 📌 Contexte du projet

Une équipe médicale souhaite explorer l'apport de l'intelligence artificielle dans le diagnostic automatisé de la **pneumonie** à partir de **radios thoraciques**.

🎯 Objectif :
Développer un **prototype fonctionnel (Proof of Concept)** capable de **classer automatiquement** une image en :
- Pneumonie
- Pas de pneumonie

Le système repose sur la **réutilisation d’un modèle pré-entraîné** de type CNN (réduction du coût d’entraînement).  
Le modèle choisi est **DenseNet121**, avec les **poids CheXNet** (entraînés sur le jeu de données ChestX-ray14), adaptés à la classification médicale.

🛠️ Suivi des expériences :  
Le projet intègre **MLflow** pour le suivi des essais, la traçabilité des modèles et les performances dans une logique **MLOps**.



- modèle à utiliser: CheXNet (ou le réseau de neuronne qui à permis de le faire : Densenet121)

## 📦 1. Chargement des bibliothèques nécessaires

Dans cette section, nous importons toutes les bibliothèques utiles pour :

- la gestion des données et des images (`numpy`,`matplotlib`, etc.),
- la construction et le chargement du modèle (`keras`),
- le suivi des expérimentations avec `MLflow`.

Nous utilisons `TensorFlow/Keras` comme framework principal pour l'entraînement.


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import random

#librairie tensorflow
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.callbacks import EarlyStopping

#librairie mlflow
import mlflow
import mlflow.tensorflow

np.set_printoptions(linewidth=np.inf)
np.set_printoptions(edgeitems=30) 

In [ ]:
IMAGE_PIXEL = (224, 224)

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            print(f"Nom du GPU détecté : {gpu.name}")
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(f"{len(gpus)} GPU(s) physique(s), {len(logical_gpus)} GPU(s) logique(s)")
        
        # Afficher les infos depuis le runtime TensorFlow
        from tensorflow.python.client import device_lib
        devices = device_lib.list_local_devices()
        for device in devices:
            if device.device_type == 'GPU':
                print("Nom      :", device.name)
                print("Type     :", device.device_type)
                print("Mémoire  :", round(device.memory_limit / (1024**3), 2), "GB")
                print("---------")
    except RuntimeError as e:
        print(e)
else:
    print("Aucun GPU détecté.")

## 🔍 2. Exploration des données

Nous examinons la structure du jeu de données en visualisant les classes présentes (ex: `PNEUMONIA`, `NORMAL`) et quelques exemples d’images.

Cela permet de vérifier que les données sont bien organisées et de se faire une idée de leur contenu avant le prétraitement.


In [ ]:
data_dir = "data/train"

classes = os.listdir(data_dir)
print("Classes disponibles :", classes)

for label in classes:
    path = os.path.join(data_dir, label)
    sample_img = random.choice(os.listdir(path))
    img = cv2.imread(os.path.join(path, sample_img), cv2.IMREAD_COLOR_RGB) #IMREAD_COLOR_RGB
    print(f"Classe : {label}")
    print(f"Type des valeurs : {img.dtype}")
    plt.imshow(img, cmap="gray")
    plt.axis("off")
    # Inspection des métadonnées de l'image
    print(f"Shape (dimensions)    : {img.shape}")
    print(f"Type des valeurs      : {img.dtype}")
    plt.show()
    # Observer l'image sous forme de matrice
    #print(img)

## 🧹 3. Préparation des données

Nous utilisons un générateur de données (`ImageDataGenerator`) pour :

- redimensionner les images à la taille attendue par le modèle (224x224),
- effectuer une normalisation simple (rescale),
- créer un split automatique entre **données d'entraînement** et **données de validation** (20%).

Les images sont converties en **RGB**, car le modèle préentraîné attend 3 canaux d’entrée.


## 🧪 4. Preprocessing

Le prétraitement est effectué automatiquement par `ImageDataGenerator`, avec les étapes suivantes :

- redimensionnement des images (224x224),
- conversion en 3 canaux (RGB),
- effectuer une normalisation simple (rescale)
- normalisation des pixels (entre 0 et 1).

Les images sont converties en **RGB**, car le modèle préentraîné attend 3 canaux d’entrée.
Ces transformations sont nécessaires pour que les images soient compatibles avec le modèle DenseNet121.

Ces modifications sont faites pour les 3 dataset qui sont:
- test
- train
- val

In [ ]:
#trier les images valides des images corrompues
# def get_valid_images(directory):
#     valid_images = []
#     labels = []

#     # Parcours des sous-dossiers (classes)
#     for class_name in os.listdir(directory):
#         class_dir = os.path.join(directory, class_name)
#         if not os.path.isdir(class_dir):
#             continue

#         for fname in os.listdir(class_dir):
#             fpath = os.path.join(class_dir, fname)
#             try:
#                 with Image.open(fpath) as img:
#                     img.verify()  # Vérifie sans charger en mémoire
#                 valid_images.append(fpath)
#                 labels.append(class_name)
#             except:
#                 print(f"Image corrompue ignorée : {fpath}")

#     return pd.DataFrame({'filename': valid_images, 'class': labels})

# # === Chemin vers ton dataset
# test_dir = 'data/train'
# train_dir = 'data/test'
# val_dir = 'data/val'

# # === Liste des images valides
# df_test = get_valid_images(test_dir)
# df_train = get_valid_images(train_dir)
# df_val = get_valid_images(val_dir)



# Prétraitement commun
datagen = ImageDataGenerator(rescale=1./255)

# Générateur pour l'entraînement
train_generator = datagen.flow_from_directory(
    'data/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    color_mode='rgb',
    shuffle=True
)

# Générateur pour le test final
test_generator = datagen.flow_from_directory(
    'data/test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    color_mode='rgb',
    shuffle=False
)

# Générateur pour la validation
val_generator = datagen.flow_from_directory(
    'data/val',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    color_mode='rgb',
    shuffle=False
)

In [ ]:
# #transformer les label en int
# label_to_int = {"NORMAL": 0, "PNEUMONIA": 1}

# def get_train_test(base_path: str, target_size=(IMAGE_PIXEL)):
#     X = []  # liste pour stocker les images
#     y = []  # liste pour stocker les étiquettes correspondantes

#     # On parcourt les sous-dossiers du répertoire (un dossier par chiffre)
#     for label in sorted(os.listdir(base_path)):
#         # on ignore les fichiers qui ne sont pas des dossiers de chiffres
#         # if not label.isdigit():
#         #     continue
#         label_path = os.path.join(base_path, label)

#         # On parcourt chaque image du dossier
#         for file_name in os.listdir(label_path):
#             file_path = os.path.join(label_path, file_name)
#             # Lecture de l'image en niveaux de gris
#             img = cv2.imread(file_path, cv2.IMREAD_COLOR_RGB)
#             if img is None:
#                 continue  # image illisible, on passe
            
#             img_resized = cv2.resize(img, target_size)
#             X.append(img_resized)           # on ajoute l'image à la liste
#             y.append(int(label_to_int[label]))    # on ajoute le label (ex: 0, 1, ..., 9)

#     # Conversion des listes en tableaux NumPy
#     X = np.array(X)
#     y = np.array(y)
#     return X, y

# X_train, y_train = get_train_test("data/train")
# X_test, y_test = get_train_test("data/test")
# X_val, y_val = get_train_test("data/test")

In [ ]:
# # Les images sont au format (28, 28). On les convertit en float pour normaliser ensuite
# X_train = X_train.astype("float32")
# X_test = X_test.astype("float32")
# X_val = X_test.astype("float32")

# # Normalisation : on divise les valeurs de pixels par 255 pour les ramener entre 0 et 1
# X_train /= 255.0
# X_test /= 255.0
# X_val /= 255.0

# # Flatten : pour un MLP, chaque image 28x28 doit devenir un vecteur de 784 valeurs
# #X_train = X_train.reshape(X_train.shape[0], 64 * 64)
# #X_test = X_test.reshape(X_test.shape[0], 64 * 64)

## 🧠 5. Modélisation

Nous utilisons le modèle **DenseNet121** disponible dans `Keras`, sans ses poids par défaut (`weights=None`), et nous le complétons avec une couche de classification binaire.

Ensuite, nous **chargeons les poids préentraînés de CheXNet**, disponibles au format `.h5`.  
Ces poids proviennent d’un entraînement sur un grand jeu de radios thoraciques (ChestX-ray14), et permettent d’adapter DenseNet121 au domaine médical.

Le modèle final est compilé pour une tâche de classification binaire (pneumonie vs normal).

## création du modèle

on reconstruit le même modele que cheXnet (pour charger les poids) car:
- Les poids .h5 sont associés à une architecture exacte
- on dois d’abord reconstruire cette même architecture (14 classes) pour les charger
- Puis, on peux couper/remplacer la dernière couche pour ton cas d’usage (binaire)

In [ ]:
# Modèle basé sur DenseNet121
input_tensor = layers.Input(shape=(IMAGE_PIXEL[0], IMAGE_PIXEL[1], 3))
model_densenet121 = DenseNet121(include_top=False, weights=None, input_tensor=input_tensor)

x = model_densenet121.output
x = layers.GlobalAveragePooling2D()(x)
output = layers.Dense(14, activation='sigmoid')(x)  # 14 sorties comme CheXNet

#output = layers.Dense(1, activation='sigmoid')(x)  # Binaire : pneumonie ou non


model = models.Model(inputs= model_densenet121.input, outputs=output)

# Charger les poids de CheXNet (Keras)
model.load_weights("weights/CheXNet_Keras_0.3.0_weights.h5")

# Retirer la dernière couche et en ajouter une nouvelle
x = model.layers[-2].output  # on reprend la couche GlobalAveragePooling2D
output = layers.Dense(1, activation='sigmoid')(x)
model = models.Model(inputs=model.input, outputs=output)

# Freeze toutes les couches du modèle (sauf la nouvelle couche Dense)
for layer in model.layers[:-1]:  # Gèle tout sauf la dernière
    layer.trainable = False

In [ ]:
# Compiler
model.compile(
    optimizer='adam',                            # Méthode d’optimisation (descente de gradient)
    loss='binary_crossentropy',                  # Fonction de perte pour classification multi-classe avec étiquettes entières (ex : 0 à 9)
    metrics=['accuracy'])                        # On surveille l’exactitude pendant l'entraînement

In [ ]:
# Résumé du modèle
model.summary()

## 📈 6. Évaluation du modèle & Suivi avec MLflow

Nous entraînons le modèle sur les radios et évaluons ses performances sur l'ensemble de validation.

Dans une logique de **traçabilité expérimentale**, nous utilisons **MLflow** pour :

- enregistrer automatiquement les métriques d'entraînement (loss, accuracy, etc.),
- suivre les paramètres du modèle,
- sauvegarder l'historique des essais pour une future comparaison ou mise en production.

Ceci initie une approche MLOps pour le suivi reproductible de nos expériences.

- lancer mlflow: mlflow ui

In [ ]:
mlflow.set_experiment("CheXNet - Pneumonia Detection")

early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)       # éviter le surapprentissage

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    callbacks=[early_stop]
)

In [ ]:
# 7. Défiger certaines couches + recompiler
for layer in model.layers[-11:]:
    layer.trainable = True

# ⚠️ Recompiler après avoir modifié trainable
model.compile(optimizer=optimizers.Adam(1e-5),  # ← plus petit LR pour ne pas détruire les poids
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Deuxième phase d'entraînement (fine-tuning)
history_fine = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    callbacks=[early_stop]
)

In [ ]:
loss, acc = model.evaluate(test_generator)
print(f"Accuracy sur le test : {acc*100:.2f}%")

In [ ]:
nommage = {0:"NORMAL", 1:"PNEUMONIA"}

# === Nombre total d'images dans le test_generator
n_images = test_generator.samples

# === Tirer un indice aléatoire entre 0 et n_images - 1
index = random.randint(0, n_images - 1)

# === Calculer le batch et la position dans le batch
batch_size = test_generator.batch_size
batch_index = index // batch_size
image_index = index % batch_size

# === Charger le batch correspondant
images_batch, labels_batch = test_generator[batch_index]

# === Récupérer l’image et le label à cet index
img = images_batch[image_index]
label = labels_batch[image_index]

# === Montrer l’image avec le label
plt.imshow(img)
plt.title(f"Label attendu : {nommage[label]}")
plt.axis('off')
plt.show()

# === Prédiction
img_input = np.expand_dims(img, axis=0)
pred = model.predict(img_input)

# === Interprétation
display(pred)